In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict, cross_val_score, KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pickle
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from datetime import datetime

In [26]:
df = pd.read_csv("yelp_dataset.csv")

In [27]:
df.rename(columns={'label':'sentiment'}, inplace=True)

In [28]:
df.shape

(40000, 7)

In [29]:
df.columns

Index(['Unnamed: 0', 'sentiment', 'reviews', 'review_after_removing_html_tags',
       'review_after_converted_emoji', 'review_after_removing_punctuation',
       'review_after_pos_and_lemmatization'],
      dtype='object')

In [30]:
df.isna().sum()

Unnamed: 0                            0
sentiment                             0
reviews                               0
review_after_removing_html_tags       0
review_after_converted_emoji          0
review_after_removing_punctuation     2
review_after_pos_and_lemmatization    4
dtype: int64

In [31]:
df['sentiment'] = df.sentiment.apply(lambda x: 0 if x == 1 else 1)

In [32]:
df[ df.sentiment == 1 ].shape

(20000, 7)

In [33]:
df[ df.sentiment == 0 ].shape

(20000, 7)

In [34]:
df.dropna(inplace=True)

In [35]:
df = df.reset_index()

In [36]:
bag_of_word_model = CountVectorizer()

X = bag_of_word_model.fit_transform(df["review_after_pos_and_lemmatization"])
y = df["sentiment"]

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [38]:
y_train[ y_train == 1 ].shape

(16157,)

In [39]:
y_train[ y_train == 0 ].shape

(15839,)

In [40]:
y_test[ y_test == 1 ].shape

(3842,)

In [41]:
y_test[ y_test == 0 ].shape

(4158,)

#### KFold cross-validation

In [42]:
# Set up k-fold cross-validation
kf = KFold(n_splits=4, shuffle=True)

random_forest_scores = []
logistic_scores = []
svm_scores = []
gd_boost_scores = []

# Generate indices for each fold
for train_index, test_index in kf.split(X):
    x_train_data, x_test_data = X[train_index], X[test_index]
    y_train_data, y_test_data = y[train_index], y[test_index]

    ## Random Forest
    model_forest = RandomForestClassifier(max_depth=10, n_estimators=500)

    print("Fitting random forest to training data....")    
    model_forest = model_forest.fit(x_train_data, y_train_data)
    
    y_pred_data = model_forest.predict(x_test_data)
    score = accuracy_score(y_test_data, y_pred_data)
    random_forest_scores.append(score)
    print(random_forest_scores)
    
    ## Logistic Regression
    
    model_logistic_regression = LogisticRegression(penalty='l1', solver='liblinear', C=0.1)
    print("Fitting Logistic Regression to training data....")  
    model_logistic_regression = model_logistic_regression.fit(x_train_data, y_train_data)
    
    y_pred_data = model_logistic_regression.predict(x_test_data)
    score = accuracy_score(y_test_data, y_pred_data)
    logistic_scores.append(score)
    print(logistic_scores)
    
    ## SVM
    model_svm = SVC(C=0.1, kernel='linear')
    print("Fitting SVM to training data....")  
    model_svm = model_svm.fit(x_train_data, y_train_data)
    
    y_pred_data = model_svm.predict(x_test_data)
    score = accuracy_score(y_test_data, y_pred_data)
    svm_scores.append(score)
    print(svm_scores)
    
    ## Gradient Boosting
    gb_classifier = GradientBoostingClassifier(random_state=2)
    print("Fitting Gradient Boosting to training data....")    
    gb_classifier.fit(x_train_data, y_train_data)
    
    y_pred_data = gb_classifier.predict(x_test_data)
    score = accuracy_score(y_test_data, y_pred_data)
    gd_boost_scores.append(score)
    print(gd_boost_scores)
    
    print("=======================================================")

# print(f"Cross-validation scores: {scores}")
# print(f"Average score: {np.mean(scores)}")

Fitting random forest to training data....
[0.8078807880788079]
Fitting Logistic Regression to training data....
[0.898989898989899]
Fitting SVM to training data....
[0.9056905690569057]
Fitting Gradient Boosting to training data....
[0.8427842784278428]
Fitting random forest to training data....
[0.8078807880788079, 0.7802780278027803]
Fitting Logistic Regression to training data....
[0.898989898989899, 0.900990099009901]
Fitting SVM to training data....
[0.9056905690569057, 0.9051905190519052]
Fitting Gradient Boosting to training data....
[0.8427842784278428, 0.8473847384738474]
Fitting random forest to training data....
[0.8078807880788079, 0.7802780278027803, 0.7774777477747775]
Fitting Logistic Regression to training data....
[0.898989898989899, 0.900990099009901, 0.8941894189418942]
Fitting SVM to training data....
[0.9056905690569057, 0.9051905190519052, 0.9014901490149015]
Fitting Gradient Boosting to training data....
[0.8427842784278428, 0.8473847384738474, 0.842284228422842

In [43]:
print("Random Forest KFold cross-validation")
print(f"Cross-validation scores: {random_forest_scores}")
print(f"Average score: {np.mean(random_forest_scores)}\n")
print("Logistic Regression KFold cross-validation")
print(f"Cross-validation scores: {logistic_scores}")
print(f"Average score: {np.mean(logistic_scores)}\n")
print("SVM cross-validation")
print(f"Cross-validation scores: {svm_scores}")
print(f"Average score: {np.mean(svm_scores)}\n")
print("Gradient Boosting KFold cross-validation")
print(f"Cross-validation scores: {gd_boost_scores}")
print(f"Average score: {np.mean(gd_boost_scores)}\n")

Random Forest KFold cross-validation
Cross-validation scores: [0.8078807880788079, 0.7802780278027803, 0.7774777477747775, 0.7844784478447845]
Average score: 0.7875287528752876

Logistic Regression KFold cross-validation
Cross-validation scores: [0.898989898989899, 0.900990099009901, 0.8941894189418942, 0.8946894689468947]
Average score: 0.8972147214721472

SVM cross-validation
Cross-validation scores: [0.9056905690569057, 0.9051905190519052, 0.9014901490149015, 0.8964896489648965]
Average score: 0.9022152215221522

Gradient Boosting KFold cross-validation
Cross-validation scores: [0.8427842784278428, 0.8473847384738474, 0.8422842284228422, 0.8380838083808381]
Average score: 0.8426342634263426



#### Random Forest

In [44]:
# Start the timer
start_time = datetime.now()

model_forest = RandomForestClassifier(max_depth=10, n_estimators=500)

print("Fitting random forest to training data....")    
model_forest = model_forest.fit(X_train, y_train)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('Random Forest Training Runtime:', runtime)

Fitting random forest to training data....
Random Forest Training Runtime: 0:01:00.958053


In [45]:
model_forest.score(X_test, y_test)

0.779625

In [46]:
model_forest.score(X_train, y_train)

0.8018189773721716

In [47]:
# Start the timer
start_time = datetime.now()

y_pred = model_forest.predict(X_test)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('Random Forest Prediction Runtime:', runtime)

Random Forest Prediction Runtime: 0:00:01.029050


In [48]:
y_pred = cross_val_predict(model_forest, X_test, y_test)

In [49]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.85      0.81      4158
           1       0.82      0.73      0.78      3842

    accuracy                           0.80      8000
   macro avg       0.80      0.79      0.79      8000
weighted avg       0.80      0.80      0.80      8000



In [50]:
print(confusion_matrix(y_test, y_pred))

[[3552  606]
 [1021 2821]]


In [51]:
filename = 'models/rd_tfidf_ml_model.sav'
pickle.dump(model_forest, open(filename, 'wb'))

#### Logistic Regression

In [52]:
# Start the timer
start_time = datetime.now()

model_logistic_regression = LogisticRegression(penalty='l1', solver='liblinear', C=0.1)

print("Fitting Logistic Regression to training data....")  
model_logistic_regression = model_logistic_regression.fit(X_train, y_train)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('Logistic Regression Training Runtime:', runtime)

Fitting Logistic Regression to training data....
Logistic Regression Training Runtime: 0:00:00.457246


In [53]:
model_logistic_regression.score(X_test, y_test)

0.90225

In [54]:
model_logistic_regression.score(X_train, y_train)

0.906957119639955

In [55]:
# Start the timer
start_time = datetime.now()

y_pred = model_logistic_regression.predict(X_test)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('Logistic Regression Prediction Runtime:', runtime)

Logistic Regression Prediction Runtime: 0:00:00.002000


In [56]:
y_pred = cross_val_predict(model_logistic_regression, X_test, y_test)

In [57]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.88      0.87      4158
           1       0.87      0.86      0.86      3842

    accuracy                           0.87      8000
   macro avg       0.87      0.87      0.87      8000
weighted avg       0.87      0.87      0.87      8000



In [58]:
print(confusion_matrix(y_test, y_pred))

[[3643  515]
 [ 533 3309]]


In [59]:
filename = 'models/lg_bow_ml_model.sav'
pickle.dump(model_logistic_regression, open(filename, 'wb'))

#### SVM

In [60]:
# Start the timer
start_time = datetime.now()

model_svm = SVC(C=0.1, kernel='linear')

model_svm = model_svm.fit(X_train, y_train)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('SVM Prediction Runtime:', runtime)

SVM Prediction Runtime: 0:08:23.954374


In [61]:
model_svm.score(X_test, y_test)

0.9055

In [62]:
model_svm.score(X_train, y_train)

0.973340417552194

In [63]:
# Start the timer
start_time = datetime.now()

y_pred = model_svm.predict(X_test)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('SVM Prediction Runtime:', runtime)

SVM Prediction Runtime: 0:00:29.042768


In [64]:
y_pred = cross_val_predict(model_svm, X_test, y_test)

In [65]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.88      0.89      4158
           1       0.87      0.88      0.88      3842

    accuracy                           0.88      8000
   macro avg       0.88      0.88      0.88      8000
weighted avg       0.88      0.88      0.88      8000



In [66]:
print(confusion_matrix(y_test, y_pred))

[[3674  484]
 [ 459 3383]]


In [67]:
filename = 'models/svm_bow_ml_model.sav'
pickle.dump(model_svm, open(filename, 'wb'))

#### Gradient Boosting Classifier

In [68]:
# Start the timer
start_time = datetime.now()


# create a Gradient Boosting classifier with default hyperparameters
gb_classifier = GradientBoostingClassifier(random_state=2)

# fit the classifier to the training data
gb_classifier.fit(X_train, y_train)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('Gradient Boosting Prediction Runtime:', runtime)


Gradient Boosting Prediction Runtime: 0:07:14.959602


In [69]:
gb_classifier.score(X_test, y_test)

0.846

In [70]:
gb_classifier.score(X_train, y_train)

0.8512001500187524

In [71]:
# Start the timer
start_time = datetime.now()

y_pred = gb_classifier.predict(X_test)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('SVM Prediction Runtime:', runtime)

SVM Prediction Runtime: 0:00:00.015662


In [72]:
y_pred = cross_val_predict(gb_classifier, X_test, y_test)

In [73]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.87      0.85      4158
           1       0.85      0.81      0.83      3842

    accuracy                           0.84      8000
   macro avg       0.84      0.84      0.84      8000
weighted avg       0.84      0.84      0.84      8000



In [74]:
filename = 'models/gb_boosting_bow_ml_model.sav'
pickle.dump(gb_classifier, open(filename, 'wb'))